# E9 Planners and their future promise

In [ ]:
!python -m pip install semantic-kernel==0.3.8.dev0
!python -m pip install chromadb

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", api_key, endpoint))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

#kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory='mymemories'))
kernel.import_skill(sk.core_skills.TextMemorySkill())

## Now let's get a 🔥 kernel ready

We're going to create a kernel that includes an embedding generation service — so that we can best construct the context that will be used.

## Note that in Example 2 I attempt to load the currently compiled SK locally

## Example 1 (skip to Example 2)

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))

In [ ]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
from semantic_kernel.planning import SequentialPlanner

kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
ask = "What day of the week is today, all uppercase?"

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

print(plan.description)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for step in plan._steps:
    print(step.description, ":", step.skill_name, step._function.name, step._state.__dict__)

print("Expected Answer:")
print(result)


# Example 2

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-4", api_key, org_id))


/Users/johnmaeda/Documents/GitHub/pizzashop-and-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "./skills"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
#text_skill = kernel.import_skill(TextSkill(), "TextSkill")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
ask = """
Tomorrow is Valentine's day. I need to come up with a poem. Translate the poem to French."""

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for step in plan._steps:
    print(step.description, ":", step.skill_name + "." + step._function.name, step._parameters._variables, step._outputs)

for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    print("Input vars:", step._parameters._variables)
    print("Output vars:", step._outputs)
    if len(step._outputs) > 0:
        print( "  Output:", str.replace(result[step._outputs[0]],"\n", "\n        "))

    print()
print("Final Answer:")
print(result)

#for step in plan._steps:
#    print(step.description, ":", step.skill_name, step._function.name, step._state.__dict__)

print("Expected Answer:")
print(result)

Summarize given text or any text document : SummarizeSkill.Summarize {'input': "Tomorrow is Valentine's day. I need to come up with a poem."} ['TOPIC']
Turn a scenario into a short and entertaining poem. : WriterSkill.ShortPoem {'input': '', 'topic': '$TOPIC'} ['POEM']
Translate the input into a language of your choice : WriterSkill.Translate {'input': '$POEM', 'language': 'French'} ['RESULT__POEM_IN_FRENCH']
Step: 0
Description: Summarize given text or any text document
Function: SummarizeSkill.Summarize
Input vars: {'input': "Tomorrow is Valentine's day. I need to come up with a poem."}
Output vars: ['TOPIC']
  Output: Valentine's day tomorrow, need poem.

Step: 1
Description: Turn a scenario into a short and entertaining poem.
Function: WriterSkill.ShortPoem
Input vars: {'input': '', 'topic': '$TOPIC'}
Output vars: ['POEM']
  Output: Roses are red, violets are blue,
        Valentine's Day is tomorrow, what to do?
        Buy chocolates, flowers, and a card,
        Or just stay hom

# Example 3

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-4", api_key, org_id))


In [ ]:
from IPython.display import display, Markdown
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "./plugins-sk"


#dt_skill = kernel.import_semantic_skill_from_directory(skills_directory, "DesignThinking")
bt_skill = kernel.import_semantic_skill_from_directory(skills_directory, "BusinessThinking")
# text_skill = kernel.import_skill(TextSkill(), "TextSkill")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
# ask = """
# Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
# Convert the text to uppercase."""
ask = """
I want to solve a business problem for a pizza shop owner who has a lot of customers who are unhappy."""

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for step in plan._steps:
    print(step.description, ":", step.skill_name, step._function.name, step._state.__dict__)

print("Expected Answer:")
print(result)



In [ ]:
ask = """
I want to solve a business problem for a pizza shop owner who has a lot of customers who are unhappy."""
original_plan = await planner.create_plan_async(ask, kernel)

Another thing the pizza shop owner is figure out how to get more time back for themself. Often times that time gained can be used for the pizza shop to reinvest in the same way that money saved can be re-leveraged.